In [ ]:
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt

--2021-10-01 03:28:06--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2661060 (2.5M) [text/plain]
Saving to: ‘fci_train_val.txt’

fci_train_val.txt   100%[===================>]   2.54M  --.-KB/s    in 0.05s   

2021-10-01 03:28:06 (56.4 MB/s) - ‘fci_train_val.txt’ saved [2661060/2661060]

--2021-10-01 03:28:06--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

In [ ]:
def read_data(filename):

    with open(filename, "r", encoding="UTF8") as f:
        data = [line.split("\t") for line in f.read().splitlines()]

    return data

In [ ]:
train_data = read_data("/content/fci_train_val.txt")
X_train_data = [t[1] for t in train_data]
y_train_data = [int(t[0]) for t in train_data]

test_data = read_data("/content/fci_test.txt")
X_test_data = [t[1] for t in test_data]
y_test_data = [int(t[0]) for t in test_data]

In [ ]:
print(len(X_train_data))
print(len(y_train_data))
print(len(X_test_data))
print(len(y_test_data))

55134
55134
6121
6121


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv("/content/fci_train_val.txt", header = None, delimiter = '\t', quoting = 3)
train_data[1]

0                        만화
1                       이치가
2                         약
3                       그사이
4                        짜긴
                ...        
55129              언제 얘기해봤어
55130              또 사기 당했어
55131                  물 좋아
55132               몇 개 먹을래
55133    오늘 오후에 올라가는 기차 취소해
Name: 1, Length: 55134, dtype: object

In [ ]:
test_data = pd.read_csv("/content/fci_test.txt", header = None, delimiter = '\t', quoting = 3)
test_data[1]

0                  연두색
1                   되면
2                    오
3                  그걸루
4                  연구소
             ...      
6116            집에 못갔어
6117        누가 너 보고 싶대
6118    쇼핑몰 근처에 병원들 많아
6119     뭐 보고 싶은 거 있어요
6120        누가 너 알고 있어
Name: 1, Length: 6121, dtype: object

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 48.8 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import re
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [ ]:
kor_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", X_train_data[13]) 
print(kor_text)

브로셔 보여준다는 게


In [ ]:
okt=Okt()
morph_text = okt.morphs(kor_text, stem=True)
print(morph_text)

['브로', '셔', '보여주다', '게']


In [ ]:
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한'])
clean_text = [token for token in morph_text if not token in stop_words]
print(clean_text)

['브로', '셔', '보여주다', '게']


In [ ]:
def preprocessing(data, okt, remove_stopwords = False, stop_words = []):
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    kor_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", data)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    morph_text = okt.morphs(kor_text, stem=True)
    
    if remove_stopwords:      
        # 불용어 제거(선택적)
        clean_text = [token for token in morph_text if not token in stop_words]
        
    return clean_text

In [ ]:
stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한', '에']
okt = Okt()
clean_train_data = []

for data in train_data[1]:
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(data) == str:
        clean_train_data.append(preprocessing(data, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_train_data.append([])  #string이 아니면 비어있는 값 추가

In [ ]:
test_data = pd.read_csv("/content/fci_test.txt", header = None, delimiter = '\t', quoting = 3)

clean_test_data = []

for data in test_data[1]:
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(data) == str:
        clean_test_data.append(preprocessing(data, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_test_data.append([])  #string이 아니면 비어있는 값 추가

In [ ]:
clean_train_data

[['만화'],
 ['이치'],
 ['약'],
 ['그', '사이'],
 ['짜다'],
 ['혜택'],
 ['지출'],
 ['어젯밤'],
 ['승진'],
 ['꼬마'],
 ['신청'],
 ['엄마', '꼭', '꼭'],
 ['어어', '그게'],
 ['브로', '셔', '보여주다', '게'],
 ['김밥'],
 ['끝내'],
 ['대학'],
 ['손잡이'],
 ['참여'],
 ['군'],
 ['차차'],
 ['집안일'],
 ['내내', '싸', '들다'],
 ['그냥'],
 ['구별'],
 ['시합'],
 ['글자'],
 ['비슷하다'],
 ['의사'],
 ['그렇다'],
 ['여군'],
 ['골목'],
 ['경쟁력'],
 ['한층'],
 ['도대체'],
 ['희곡'],
 ['이르다'],
 ['없다'],
 ['반죽'],
 ['자리'],
 ['지금', '도착'],
 ['볶음밥'],
 ['그거'],
 ['참석자'],
 ['젓가락'],
 ['소위'],
 ['부'],
 ['몇몇'],
 ['얘길'],
 ['너무'],
 ['발음'],
 ['또'],
 ['역'],
 ['아나운서'],
 ['북'],
 ['걸음'],
 ['단둘'],
 ['햄'],
 ['의사'],
 ['실수'],
 ['치아'],
 ['개미'],
 ['뿌듯'],
 ['처음'],
 ['누구'],
 ['영화배우'],
 ['여친'],
 ['버터'],
 ['비참하다'],
 ['우리'],
 ['오로지'],
 ['절'],
 ['얘길'],
 ['실내'],
 ['단순하다'],
 ['과제'],
 ['좌석'],
 ['여럿'],
 ['주변'],
 ['한번'],
 ['뭐', '야', '그거'],
 ['나쁘다', '눔'],
 ['최소한'],
 ['형수'],
 ['방면'],
 ['영역'],
 ['암'],
 ['종'],
 ['그만'],
 ['국제선'],
 ['안녕'],
 ['과외'],
 ['담'],
 ['유리창'],
 ['퇴직금'],
 ['위아래'],
 ['과자'],
 ['아깝다'],
 ['여기'],
 ['금메달'],
 ['그렇다', '거

In [ ]:
clean_test_data

[['연두색'],
 ['되다'],
 ['오'],
 ['그걸루'],
 ['연구소'],
 ['접근'],
 ['출퇴근'],
 ['갑자기'],
 ['환경오염'],
 ['태도'],
 ['깨달음'],
 ['창피하다'],
 ['자주'],
 ['넷'],
 ['아저씨', '제'],
 ['높이'],
 ['뜻밖'],
 ['과정'],
 ['운전사'],
 ['선재'],
 ['월세'],
 ['그냥', '빌다'],
 ['뭔가'],
 ['세'],
 ['무렵'],
 ['예순'],
 ['탁구'],
 ['있다'],
 ['우리'],
 [],
 ['토마토'],
 ['샌드위치'],
 ['땀'],
 ['대도시'],
 ['도저히'],
 ['침'],
 ['딱'],
 ['뭐', '하다'],
 ['가난'],
 ['거'],
 ['똑바로'],
 ['설명'],
 ['술', '퍼', '말다'],
 ['칼'],
 ['행사'],
 ['연구'],
 ['그렇다'],
 ['공짜'],
 ['그냥'],
 ['팔십'],
 ['내', '봄'],
 ['속담'],
 ['얘기', '하고'],
 ['감정'],
 ['셋'],
 ['기초', '적'],
 ['요리'],
 ['양심'],
 ['잠자리'],
 ['지금껏'],
 ['지적'],
 ['시청'],
 ['깨끗이'],
 ['목록'],
 ['결심'],
 ['건조'],
 ['두', '박'],
 ['마음'],
 ['긴장'],
 ['수학'],
 ['취향'],
 ['하다', '번'],
 ['이제'],
 ['센티미터'],
 ['전철'],
 ['하나'],
 ['왜'],
 ['떨다'],
 ['네', '년'],
 ['꽤'],
 ['국립'],
 ['만날'],
 ['가르침'],
 ['마무리'],
 ['앞서'],
 ['웨이터'],
 ['근무'],
 ['식초'],
 ['못', '하다'],
 ['전화기'],
 ['그렇다'],
 ['나'],
 ['장비'],
 ['요금'],
 [],
 ['로또'],
 ['마요네즈'],
 ['제공'],
 ['당신'],
 ['얘기', '냐'],
 ['도로'],
 ['주인'],
 ['칠월']

In [43]:
def write_txt(list, fname, sep) :
    file = open(fname, 'w')
    vstr = ''
    
    for a in list :
        vstr = vstr + str(a) + sep
    vstr = vstr.rstrip(sep)  # 마지막에도 추가되는  sep을 삭제 
    
    file.writelines(vstr)      # 한 라인씩 저장 
        
    file.close()
    print('[ 파일 저장 완료 ]')

In [44]:
write_txt(clean_train_data, 'clean_train_data.txt', sep=',')

[ 파일 저장 완료 ]


In [45]:
write_txt(clean_test_data, 'clean_test_data.txt', sep=',')

[ 파일 저장 완료 ]
